In [ ]:
import os
import json

tweetFolder = '../data/tweets/'
tweetFiles = os.listdir(tweetFolder)

info = []

In [ ]:
for tf in tweetFiles:
    with open(tweetFolder + tf, 'r') as f:
        info.append(json.load(f))
        
dict(info[0])

In [ ]:
info[1]['entities']

In [ ]:
for t in info:
    print('------------------')
    print(t['full_text'])
    print('---')
    hl = []
    if t['entities']['hashtags']:
        hl = [h['text'] for h in t['entities']['hashtags']]
    print('Hashtags: ', ' '.join(hl))
    
    ul = []
    if t['entities']['urls']:
        ul = [u['expanded_url'] for u in t['entities']['urls']]
    print('Urls: ', ' '.join(ul))